In [30]:
import numpy as np
import random
from pebble import ThreadPool

import time

def generate_3sat_formula(n, m):
    """Generate a random 3-SAT formula with n variables and m clauses."""
    formula = []
    valid_assignment = [False] + [random.random() < 0.5 for _ in range(n)]
    while len(formula) < m:
        clause = random.sample(range(1, n + 1), 3)
        clause = [var if random.random() < 0.5 else -var for var in clause]
        # Check if the clause is satisfied by the valid assignment
        if any((valid_assignment[abs(lit)] > 0) == (lit > 0) for lit in clause):
            formula.append(clause)
    return formula

def check_3sat_formula(formula, assignment):
    print(formula, assignment)
    return all(any((assignment[abs(lit)] > 0) == (lit > 0) for lit in clause) for clause in formula)

def utility(algorithm_str: str):
    """
    Implements the Random 3-SAT problem with n variables and m clauses.
    Returns the fraction of formulas solved successfully within the time limit.
    The algorithm must be extremely fast. If it takes more than 10 milliseconds to run, it is a failure.
    Your algorithm function must be named ’algorithm’ and take a single argument, formula which is a list of m clauses, each containing exactly 3 literals.
    """
    n_tests = 100
    n = 50 # Number of variables
    m = int(4 * n) # Number of clauses
    solved_count = 0
    pool = ThreadPool()

    try:
        exec(algorithm_str, globals())
    except:
        return 0
    print(n_tests)
    for test_idx in range(2):
        print(test_idx)
        formula = generate_3sat_formula(n, m)
        print(formula)
        try:
            assignment_future = pool.schedule(algorithm, (formula,))
            assignment = assignment_future.result(timeout=0.01)
            print(formula, assignment)
            if check_3sat_formula(formula, assignment):
                solved_count += 1
        except Exception as e:
            return 0

    return solved_count / n_tests


algorithm_str = """import random

def random_walk_solver(formula, max_iter, p):
    n = max(abs(lit) for clause in formula for lit in clause)

    assignments = [False] * (n + 1)
    for _ in range(max_iter):
        unsatisfied_clauses = [clause for clause in formula if not any(assignments[abs(lit)] == (lit > 0) for lit in clause)]
        if not unsatisfied_clauses:
            return assignments
        clause_to_flip = random.choice(unsatisfied_clauses)
        if random.random() < p:
            lit_to_flip = random.choice(clause_to_flip)
        else:
            lit_to_flip = min(clause_to_flip, key=lambda lit: sum(assignments[abs(lit)] == (lit > 0) for clause in formula if lit in clause))
        assignments[abs(lit_to_flip)] = not assignments[abs(lit_to_flip)]
    return None

def algorithm(formula):
    return random_walk_solver(formula, max_iter=1000, p=0.4)"""


print(utility(algorithm_str))


100
0
[[28, 18, 30], [-25, 43, -22], [19, -24, 4], [42, 6, 10], [-21, -9, 6], [42, 26, 37], [12, -36, -42], [7, -35, -41], [44, -2, 4], [16, 15, -21], [15, -28, 1], [26, -20, -7], [-30, -50, -24], [-33, -38, -36], [-13, -38, 3], [-4, 22, 3], [-24, -11, 34], [10, -21, 20], [-10, 19, -25], [23, 28, -13], [12, 29, 19], [-18, 9, -43], [34, 21, 23], [-20, 32, -21], [15, -9, 37], [16, -50, -25], [-9, -26, -44], [-46, -19, 8], [31, 6, 34], [-9, -47, 4], [21, 31, 35], [23, 45, -38], [18, 35, 21], [48, 2, -16], [-8, -26, 23], [-17, -4, 49], [-16, 33, -5], [-28, 44, -6], [-2, -25, -7], [-32, 23, -7], [-11, 1, 32], [21, -26, 9], [-15, -47, -3], [39, -23, 28], [22, 40, -12], [-2, -40, 20], [5, -41, -45], [11, 44, 43], [-41, 42, 31], [7, 13, -23], [34, -46, -20], [-38, 22, -44], [26, 21, 41], [-26, 39, 37], [27, 16, -1], [-30, -1, 36], [32, -8, 25], [9, -19, 2], [1, 21, -23], [-44, -9, 27], [38, 5, 50], [-42, 16, -44], [-35, -26, 20], [-7, 48, -24], [35, -33, -1], [10, -33, 41], [-48, -46, -20], [-